This notebook demonstrates how to use the `HelmholtzSolver` class. First, the object has to be initiated. Then, the required coefficients can be computed using the `compute_coefficients` method of the class. The method returns:
- $R$
- $|R|^2$
- $T$
- $|T|^2$
- $A = 1 - |R|^2 - |T|^2$
where the coefficients are defined according to the asymptotic conditions: 

$$
E(z) \rightarrow 
\begin{cases}
    e^{ik_0z}+Re^{-ik_0z}, & z \rightarrow -\infty \\
    Te^{ik_0z}, & z \rightarrow +\infty
\end{cases}
$$

The slab structure is defined by providing a list coordinates of the interfaces and a list defining a piecewise dielectric permittivity. 

**Units**:
- $z$, $\lambda$: meters/micrometers, etc
- $\theta$: deg

In [7]:
from HelmholtzSolver import HelmholtzSolver
import numpy as np

def format_complex(z):
    # Разделяем действительную и мнимую части
    real = z.real
    imag = z.imag
    # Форматируем с фиксированной шириной и обязательным знаком
    return f"{real:+.4f}{imag:+.4f}j"

def print_coefficients(R, T):
    R_str = format_complex(R)
    T_str = format_complex(T)
    R_pow = f"{np.abs(R)**2:.5f}"
    T_pow = f"{np.abs(T)**2:.5f}"
    A = f"{1 - np.abs(R)**2 - np.abs(T)**2:.5f}"

    print(f"  R = {R_str:<16} |R|^2 = {R_pow:>8}")
    print(f"  T = {T_str:<16} |T|^2 = {T_pow:>8}")
    print(f"  A = {A:>7}")

### Example

In [ ]:
eps = -10 + 20j
solver = HelmholtzSolver(
    lambda0=1,
    boundaries=[0, 200, 400, 600],
    epsilons=[eps, eps, eps],
    theta=0,
    polarization="s"
)
R, T, A = solver.compute_coefficients()

print_coefficients(R, T)

5054.796593569815 (2513.2741228718346+0j)
  R = -0.7539-0.2839j  |R|^2 =  0.64902
  T = +0.0000+0.0000j  |T|^2 =  0.00000
  A = 0.35098


In [6]:
a = 800
b = 400

print(np.exp(a)/np.exp(b), np.exp(a-b))



inf 5.221469689764144e+173


/tmp/ipykernel_12105/4159062700.py:4: RuntimeWarning: overflow encountered in exp
  print(np.exp(a)/np.exp(b), np.exp(a-b))


In [9]:
eps = -10 - 20j
z = 1j*10*eps

print(np.exp(z.real, dtype=np.float128) * (np.cos(z.imag, dtype=np.float128) + 1j*np.sin(z.imag, dtype=np.float128)), np.exp(-z.real, dtype=float), np.exp(z, dtype=np.complex128), np.exp(z, dtype=complex))

(6.2310935509105822157e+86+3.6589848397392945553e+86j) 1.3838965267367376e-87 (6.231093550910581e+86+3.6589848397392945e+86j) (6.231093550910581e+86+3.6589848397392945e+86j)


### Absorption

In [3]:
epsilon = 4 + 1j  
solver = HelmholtzSolver(
    lambda0=1.0,
    boundaries=[0, 0.1],
    epsilons=[epsilon],
    theta=0,  
)
R, T, A = solver.compute_coefficients()
assert A > 0, f"Ошибка: A = {A} (должно быть > 0)"
print(f"A = {A:.4f}")

[[ 1.50766473+0.1240492j -0.50766473-0.1240492j]
 [-0.50766473-0.1240492j  1.50766473+0.1240492j]] [[ 1.50766473+0.1240492j -0.50766473-0.1240492j]
 [-0.50766473-0.1240492j  1.50766473+0.1240492j]]
[[ 1.50766473+0.1240492j -0.50766473-0.1240492j]
 [-0.50766473-0.1240492j  1.50766473+0.1240492j]] [[ 0.66662565-1.63745373j -0.02898788-0.44622692j]
 [-0.31620232+0.52253512j  0.28555038+1.26251477j]]
A = 0.2314


### Normal incidence (s equals to p)

In [7]:
epsilon = 2.25
solver_s = HelmholtzSolver(
    lambda0=1.0,
    boundaries=[0, 0.1],
    epsilons=[epsilon],
    theta=0,
    polarization="s"
)
R_s, T_s, A_s = solver_s.compute_coefficients()

solver_p = HelmholtzSolver(
    lambda0=1.0,
    boundaries=[0, 0.1],
    epsilons=[epsilon],
    theta=0,
    polarization="p"
)
R_p, T_p, A_p = solver_p.compute_coefficients()

assert np.isclose(abs(R_s), abs(R_p), atol=1e-6), f"Ошибка: |R_s|={abs(R_s)}, |R_p|={abs(R_p)}"

[[ 0.73473157-1.01127124j -0.14694631-0.20225425j]
 [-0.14694631+0.20225425j  0.73473157+1.01127124j]]
[[0.48982104-0.67418083j 0.09796421+0.13483617j]
 [0.09796421-0.13483617j 0.48982104+0.67418083j]]


### No layers

In [8]:
epsilon = 4 + 1j  
solver = HelmholtzSolver(
    lambda0=1.0,
    boundaries=[0, 0],
    epsilons=[15],
    theta=0,  
)
R, T, A = solver.compute_coefficients()

assert np.isclose(abs(R), 0, atol=1e-6)

[[ 2.43649167+0.j -1.43649167+0.j]
 [-1.43649167+0.j  2.43649167+0.j]]


In [9]:
k0 = 1
k1 = 20j
k = k0 + k1

d = 100
f = np.exp(-1j*k*d)

f1 = np.exp(k.imag*d)*(np.cos(k0*d)+1j*np.sin(k0*d))
print(np.exp(k.imag*d))
print(f, f1)

inf
(inf+infj) (inf-infj)


/tmp/ipykernel_8009/2812407858.py:6: RuntimeWarning: overflow encountered in exp
  f = np.exp(-1j*k*d)
/tmp/ipykernel_8009/2812407858.py:8: RuntimeWarning: overflow encountered in exp
  f1 = np.exp(k.imag*d)*(np.cos(k0*d)+1j*np.sin(k0*d))
/tmp/ipykernel_8009/2812407858.py:9: RuntimeWarning: overflow encountered in exp
  print(np.exp(k.imag*d))


In [18]:
def safe_exp(x):
    x_real = np.real(x)
    x_imag = np.imag(x)
    x_real_clipped = np.clip(x_real, -700, 700)  # Ограничиваем вещественную часть
    return np.exp(x_real_clipped + 1j*x_imag)

safe_exp(800)

np.complex128(1.0142320547350045e+304+0j)